In [20]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_log_error, mean_squared_error,mean_absolute_error



In [7]:
data=pd.read_pickle("./dataframe/df_only_risky_events.pkl")
data.reset_index(inplace=True)
data.drop(['index'], inplace=True, axis=1)

In [8]:
train, test = train_test_split(data, test_size=0.30, random_state=42)
print("Train dataframe dimension {} x {}".format(train.shape[0],train.shape[1]))
print("Test dataframe dimension {} x {}".format(test.shape[0],test.shape[1]))
Y_train = train["COLLISSION_PROBABILITY"]
X_train= train.drop(["COLLISSION_PROBABILITY"], axis=1)
Y_test = test["COLLISSION_PROBABILITY"]
X_test= test.drop(["COLLISSION_PROBABILITY"], axis=1)

Train dataframe dimension 190 x 86
Test dataframe dimension 82 x 86


In [ ]:
# Y = df["COLLISSION_PROBABILITY"]
# X = df.drop(["COLLISSION_PROBABILITY"], axis=1)

In [9]:
ncol=X_train.shape[1]

In [10]:
def baseline_model():
    # create model
	model = Sequential()
	model.add(Dense(ncol, input_dim=ncol, kernel_initializer='normal', activation='relu'))
	model.add(Dense(ncol, kernel_initializer='normal'))
	# Compile model
	model.compile(loss='mean_squared_error', optimizer='adam')
	return model

In [11]:
# evaluate model
estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=10)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

/tmp/ipykernel_14498/1496378904.py:2: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
2021-11-29 22:17:44.050655: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-11-29 22:17:44.050697: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-29 22:17:44.050728: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (acer): /proc/driver/nvidia/version does not exist


Baseline: -1241113525154.00 (3613428639264.20) MSE


In [18]:
# Convert into Numpy array
samples_to_predict = np.array(X_test)

# Generate predictions for samples
predictions = baseline_model().predict(samples_to_predict)

[[-4.6100328e+05 -1.0118496e+04 -2.2740434e+05 ...  3.1287175e+05
  -2.9365019e+05  5.5086875e+03]
 [-3.5896559e+05 -7.8521348e+03 -1.7700838e+05 ...  2.4318266e+05
  -2.2841730e+05  4.3233008e+03]
 [-7.1781815e+06 -1.4698778e+05 -3.5488488e+06 ...  4.8832640e+06
  -4.5815970e+06  8.7155000e+04]
 ...
 [-1.4230889e+05 -3.8034111e+03 -7.0003398e+04 ...  9.5515000e+04
  -8.9830484e+04  1.5978125e+03]
 [-6.0426094e+04 -1.5592573e+03 -2.9299502e+04 ...  4.0671938e+04
  -3.8239227e+04  8.0019336e+02]
 [-1.7707033e+02 -1.5145264e+01  1.1223202e+02 ...  3.7580349e+01
   7.5259190e+00  1.0126320e+02]]


In [24]:
Y_pred = pd.DataFrame(predictions)

In [25]:
Y_pred

,0,1,2,3,4,5,6,7,8,9,...,75,76,77,78,79,80,81,82,83,84
0,-4.610033e+05,-10118.496094,-2.274043e+05,1.821591e+05,-18714.894531,-6.482261e+04,3.109068e+05,-4.064199e+05,-4.648611e+04,-5.374986e+05,...,5.605469e+05,-6987.493164,9.331230e+04,-2.577813e+05,-1.137224e+05,3.605360e+05,-3.185858e+05,3.128718e+05,-2.936502e+05,5508.687500
1,-3.589656e+05,-7852.134766,-1.770084e+05,1.416559e+05,-14510.966797,-4.990900e+04,2.415592e+05,-3.160592e+05,-3.540011e+04,-4.188578e+05,...,4.367406e+05,-5074.024414,7.309503e+04,-1.999858e+05,-8.807481e+04,2.807604e+05,-2.479054e+05,2.431827e+05,-2.284173e+05,4323.300781
2,-7.178182e+06,-146987.781250,-3.548849e+06,2.836030e+06,-291008.000000,-1.006685e+06,4.831710e+06,-6.329087e+06,-7.140054e+05,-8.374766e+06,...,8.732084e+06,-105143.484375,1.456371e+06,-4.001235e+06,-1.758162e+06,5.611039e+06,-4.958682e+06,4.883264e+06,-4.581597e+06,87155.000000
3,-4.490455e+05,-9586.560547,-2.215449e+05,1.775354e+05,-17790.609375,-6.298771e+04,3.029705e+05,-3.962183e+05,-4.485433e+04,-5.241582e+05,...,5.462478e+05,-6775.270508,9.063627e+04,-2.507983e+05,-1.103341e+05,3.511869e+05,-3.106527e+05,3.050360e+05,-2.863832e+05,5445.242188
4,-1.076465e+04,-3079.177246,9.847783e+02,5.964138e+03,-5955.022461,-1.536927e+04,1.216793e+04,-5.597261e+03,-6.910667e+03,-8.193601e+03,...,7.191292e+03,-3177.169189,-2.682387e+03,-1.611261e+04,-8.530089e+03,7.800210e+03,-8.698322e+03,4.170927e+03,-4.988652e+03,4371.220703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,-1.328930e+07,-271081.562500,-6.572286e+06,5.251121e+06,-536453.750000,-1.863098e+06,8.945340e+06,-1.171562e+07,-1.321069e+06,-1.550772e+07,...,1.616946e+07,-194652.218750,2.696046e+06,-7.406483e+06,-3.254999e+06,1.038809e+07,-9.181150e+06,9.040519e+06,-8.484158e+06,159341.250000
78,-1.267941e+05,-2800.457031,-6.232670e+04,4.998281e+04,-5456.924805,-1.801221e+04,8.534364e+04,-1.119400e+05,-1.272064e+04,-1.474222e+05,...,1.537297e+05,-1897.647217,2.560168e+04,-7.088012e+04,-3.118321e+04,9.914391e+04,-8.745861e+04,8.606563e+04,-8.057480e+04,1815.507812
79,-1.423089e+05,-3803.411133,-7.000340e+04,5.625019e+04,-5476.355469,-1.986033e+04,9.641595e+04,-1.252840e+05,-1.469947e+04,-1.660237e+05,...,1.733298e+05,-1962.099121,2.867119e+04,-8.019757e+04,-3.563191e+04,1.116768e+05,-9.892466e+04,9.551500e+04,-8.983048e+04,1597.812500
80,-6.042609e+04,-1559.257324,-2.929950e+04,2.402442e+04,-2062.301758,-8.419811e+03,4.123842e+04,-5.357957e+04,-6.172764e+03,-7.069676e+04,...,7.330170e+04,-1015.040649,1.172596e+04,-3.407288e+04,-1.494848e+04,4.733425e+04,-4.210920e+04,4.067194e+04,-3.823923e+04,800.193359


In [21]:
    # REGRESION MODEL METRICS
    print("The r2 of prediction is:", r2_score(Y_test, Y_pred))
    print("The MSE of prediction is:", mean_squared_error(Y_test, Y_pred, squared=True))
    print(
        "The RMSE of prediction is:", mean_squared_error(Y_test, Y_pred, squared=False)
    )
    print("The MAE of prediction is:", mean_absolute_error(Y_test, Y_pred))

ValueError: y_true and y_pred have different number of output (1!=85)